In [1]:
import pandas as pd
import requests
import gmaps
from config import google_maps_api_key

In [2]:
city_data_df=pd.read_csv("weather_data/cities.csv")
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [3]:
city_data_df.head(2)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Deputatskiy,RU,2020-02-20 16:57:08,69.30,139.90,-5.33,96,100,3.42
1,1,Rikitea,PF,2020-02-20 16:57:09,-23.12,-134.97,79.03,71,71,8.55


In [4]:
gmaps.configure(api_key=google_maps_api_key)

In [5]:
# Create the city location variable
locations = city_data_df[["Lat", "Lng"]]
# Create the variable to use as weight
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
# Create the figure in gmaps
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Create a heat layer variable to later add to figure
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add heat layer to figure
fig.add_layer(heat_layer)
# Call figure from gmaps with heat layer on top
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
locations = city_data_df[["Lat", "Lng"]]
cloudiness = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=cloudiness, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna(axis=0, how="any")
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Rikitea,PF,2020-02-20 16:57:09,-23.12,-134.97,79.03,71,71,8.55
2,2,Castro,BR,2020-02-20 16:57:09,-24.79,-50.01,83.97,46,31,6.55
4,4,Mahebourg,MU,2020-02-20 16:57:10,-20.41,57.70,78.80,78,20,3.36
8,8,Bambous Virieux,MU,2020-02-20 16:57:12,-20.34,57.76,78.80,78,20,3.36
13,13,Atuona,PF,2020-02-20 16:57:15,-9.80,-139.03,81.52,76,27,21.16


In [11]:
preferred_cities_df.count()

City_ID       156
City          156
Country       156
Date          156
Lat           156
Lng           156
Max Temp      156
Humidity      156
Cloudiness    156
Wind Speed    156
dtype: int64

In [12]:
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Rikitea,PF,79.03,-23.12,-134.97,
2,Castro,BR,83.97,-24.79,-50.01,
4,Mahebourg,MU,78.80,-20.41,57.70,
8,Bambous Virieux,MU,78.80,-20.34,57.76,
13,Atuona,PF,81.52,-9.80,-139.03,
15,Lorengau,PG,83.37,-2.02,147.27,
16,Carnarvon,AU,82.40,-24.87,113.63,
17,Cabo San Lucas,MX,80.01,22.89,-109.91,
23,Luanda,AO,87.80,-8.84,13.23,
24,Lebu,CL,79.65,-37.62,-73.65,


In [20]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_maps_api_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [21]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Rikitea,PF,79.03,-23.12,-134.97,Pension Maro'i
2,Castro,BR,83.97,-24.79,-50.01,CHACARA BAILLY
4,Mahebourg,MU,78.80,-20.41,57.70,Shandrani Beachcomber Resort & Spa
8,Bambous Virieux,MU,78.80,-20.34,57.76,Casa Tia Villa
13,Atuona,PF,81.52,-9.80,-139.03,Villa Enata
15,Lorengau,PG,83.37,-2.02,147.27,Lorengau Harbourside Hotel
16,Carnarvon,AU,82.40,-24.87,113.63,Hospitality Carnarvon
17,Cabo San Lucas,MX,80.01,22.89,-109.91,Hotel Tesoro Los Cabos
23,Luanda,AO,87.80,-8.84,13.23,EPIC SANA Luanda
24,Lebu,CL,79.65,-37.62,-73.65,Hostal Las Lilas


In [24]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [25]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))